In [ ]:
# Install required libraries
!pip install PyPDF2
!pip install faiss-cpu
!pip install langchain
!pip install transformers

In [ ]:
import time
from PyPDF2 import PdfReader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from transformers import pipeline

In [ ]:
# Path to the PDF file
pdf_path = "/content/EW.pdf"

In [ ]:
# Read PDF and extract text
pdf = PdfReader(pdf_path)
raw_text = ''
for i, page in enumerate(pdf.pages):
    content = page.extract_text()
    if content:
        raw_text += content

In [ ]:
# Split the text into chunks
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=800,
    chunk_overlap=200,
    length_function=len,
)
texts = text_splitter.split_text(raw_text)

In [ ]:
# Initialize Hugging Face Embeddings
model_id = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_id)

<ipython-input-34-eb5a3409aa1b>:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_id)
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Generate embeddings for text chunks and store in FAISS vector store
document_store = FAISS.from_texts(texts, embedding=embeddings)

In [ ]:
# Load a question-answering pipeline from Hugging Face
qa_model_id = "deepset/roberta-base-squad2"  # You can use any compatible QA model
qa_pipeline = pipeline("question-answering", model=qa_model_id, tokenizer=qa_model_id)

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
# Function to handle queries
def query_pdf(query):
    # Search for similar documents in the FAISS vector store
    docs = document_store.similarity_search(query)
    # Combine document texts
    input_text = " ".join(doc.page_content for doc in docs)
    # Use the QA pipeline to get an answer
    answer = qa_pipeline(question=query, context=input_text)
    return answer['answer']

In [ ]:
# Example query
query = input("Enter your query: ")
answer = query_pdf(query)
print(f"Answer: {answer}")

Enter your query: what is engineering graphics
Answer: an  effective way  of communicating technical ideas


In [ ]:
# Example query
query = input("Enter your query: ")
answer = query_pdf(query)
print(f"Answer: {answer}")

Enter your query: what is revision table
Answer: normally located in the upper right of the drawing 
frame


In [ ]:
# Example query
query = input("Enter your query: ")
answer = query_pdf(query)
print(f"Answer: {answer}")

Enter your query: what is orthographic projection
Answer: a means of representing a 
three -dimensional object in two dimensions
